## DoorDash Stock Analysis (Cleaning Version)

Import the basic function package `pandas`, `numpy`, `matplotlib.pyplot` and load the data file `DASH_A1.csv`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("DASH_A1.csv")
df

## Data Viewing and Preprocessing
For easy reading, change the format of the `Date` and thus can be quickly indexed.

In [ ]:
df.Date = pd.to_datetime(df.Date, format="%d-%m-%Y")
df.set_index("Date", inplace=True)
df


In [ ]:
# Sort the table by date in ascending order
df.index.is_monotonic_increasing # check whether the current data is in the correct order
df.sort_index(inplace=True)  
df


In [ ]:
# check whether there is duplicated data, if so, drop them
df.duplicated().sum() # there are 50 duplicated datas

df.drop_duplicates(inplace=True) 
df

df.duplicated().sum() 


In [ ]:
# check the total number of missing values in the dataframe for each column 
df.isnull().sum()

There are **23** missing value "Close"; **24** missing value "High", **15** missing value "Low", **15** missing value "Open", **26** missing value "Volume".

Then can find out which rows have missing data by using `isnull()`, `any()` along rows.

In [ ]:
condition = df.isnull().any(axis=1)
df[condition]

## Data Cleaning
Since all 5 columns have some missing values, we then use different approaches to fill the missing values.

### Open Price
Fill missing values in the Open column **with the Close of the day before** as an approximation, ignoring overnight trading.

In [ ]:
df['Open'] = df['Open'].fillna(df['Close'].shift(1))
df.isnull().sum()

### Close Price
Fill missing values in the Close column with the methods **forward-filled** to avoid look-ahead bias.

In [ ]:
df['Close'] = df['Close'].ffill()
df.isnull().sum()

### High and Low Price
Fill missing values in the High and Low columns **with the mean of the respective High or Low within that month**.

In [ ]:
df["Month"] = df.index.to_period("M")
df

df["High"] = df["High"].fillna(df.groupby("Month")["High"].transform("mean"))
df["Low"] = df["Low"].fillna(df.groupby("Month")["Low"].transform("mean"))
df.isnull().sum()

#for col in ["High", "Low"]:
    #for period in df["Month"].unique():
        #mask = df["Month"] == period
        #monthly_mean = df.loc[mask, col].mean()
        #df.loc[mask, col] = df.loc[mask, col].fillna(monthly_mean)


### Volume (Zero Volume Condition)
**If the Close price is equal to the Open price, fill the missing values in the Volume column with zero**, indicating
no change in trading activity for the day.

In [ ]:
condition_equal = (df.Volume.isnull()) & (df.Close == df.Open)
df.loc[condition_equal, "Volume"] = 0

df.isnull().sum()

### Volume (Non-zero Volume Condition)
**If the Close price is not equal to the Open price, fill the missing values in the Volume column with the median of the existing Volume values**.

In [ ]:
condition_notequal = (df['Volume'].isnull()) & (df['Close'] != df['Open']) 
df.loc[condition_notequal, 'Volume'] = df['Volume'].median()

df.isnull().sum()

In [ ]:
df.isnull().sum().sum() # double check there is no missing value
df.info()

## Feature Engineering 

### Simple Daily Returns
Calculate the simple daily returns to measure the day-to-day percentage change in the Close prices.

In [ ]:
df["SimpleDailyReturns"] = df.Close.pct_change()
df

### Logarithmic Returns
Calculate the logarithmic returns using Close prices.

In [ ]:
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
df

### 20-Day Momentum
Calculate the 20-day momentum by subtracting the Close price 20 days prior from the current Close price, providing insights into the stock's short-term trend.

In [ ]:
df['20Day_Momentum'] = df['Close'] - df['Close'].shift(20)
df

### 20-Day Simple Moving Average
Calculate the 20-day simple moving average to smooth out short-term fluctuations and highlight longer-term trends in the Close prices.

In [ ]:
df['20Day_SMA'] = df['Close'].rolling(window=20).mean()
df

### 20-Day Rolling Volatility
Calculate the 20-day rolling volatility based on the standard deviation of simple daily returns to indicate the stock's risk level.

In [ ]:
df['20Day_Volatility'] = df['SimpleDailyReturns'].rolling(window=20).std()
df

### Day of the Week
Identify the day of the week for each trading day.

In [ ]:
df['Day_of_Week'] = df.index.day_name()
df

### Price Surge Identification: 
Identify days where the price surged, defined as when the daily return is more than 4 standard deviations above the mean daily return for the period, indicating significant price movements.

In [ ]:
# Criteria: Simple Return > Mean + 4 * Std Dev
mean_return = df['SimpleDailyReturns'].mean()
std_return = df['SimpleDailyReturns'].std()
df['Price_Surge'] = df['SimpleDailyReturns'] > (mean_return + 4 * std_return)
df

surge_days = df[df['Price_Surge'] == True]
surge_days

### Volume Spike Identification
Identify days where the volume spiked, defined as when the trading volume is more than 6 standard deviations above the mean volume for the period, highlighting unusual trading activity.

In [ ]:
# Criteria: Volume > Mean + 6 * Std Dev
mean_volume = df['Volume'].mean()
std_volume = df['Volume'].std()
df['Volume_Spike'] = df['Volume'] > (mean_volume + 6 * std_volume)

unusualtrading_days = df[df['Volume_Spike'] == True]
unusualtrading_days

### Bollinger Bands Calculation
Calculate the upper and lower Bollinger Bands for the stock, which are set at 2 standard deviations above and below the 20- day simple moving average, to identify overbought and oversold conditions.

In [ ]:
df['Dev'] = df['Close'].rolling(window=20).std()

df['Upper_BB'] = df['20Day_SMA'] + 2 * df['Dev']
df['Lower_BB'] = df['20Day_SMA'] - 2 * df['Dev']
df

In [ ]:
plt.figure(figsize=(8,4))
plt.title("Bollinger on DoorDash")
plt.xlabel("Date")
plt.ylabel("Price(USD)")

plt.plot(df.Upper_BB, label="Upper Bollinger Band", color="grey", linestyle="--")
plt.plot(df.Lower_BB, label="Lower Bollinger Band", color="grey",linestyle="--")
plt.fill_between(df.index, df.Upper_BB, df.Lower_BB, color="grey", alpha=0.5) 
plt.plot(df.Close, label="Closing Price")
plt.legend()
plt.tight_layout()

## Key Dates



### Highest Volatility Day

In [ ]:
# Ensure the Year column exists
df['Year'] = df.index.year

# Remove missing values in 20-day volatility
df_vol = df.dropna(subset=['20Day_Volatility'])

# Find the index of the max volatility in each year
date_vol_max = df_vol.groupby('Year')['20Day_Volatility'].idxmax()


# Select those rows from the original dataframe
highest_vol_days = df.loc[date_vol_max, ['Year', '20Day_Volatility']]
# Display the results
print("Highest Volatility Day per Year:")
print(highest_vol_days)

### Largest Price Surge Day

In [ ]:
# Ensure the Year column exists
df['Year'] = df.index.year

# Remove missing values in simple return
df_ret = df.dropna(subset=['SimpleDailyReturns'])

# Get the index of the highest daily return in each year
date_surge_max = df_ret.groupby('Year')['SimpleDailyReturns'].idxmax()

# Retrieve those rows
highest_surge_days = df.loc[date_surge_max, ['Year', 'SimpleDailyReturns']]

# Display the results
print("Largest Price Surge Day per Year:")
print(highest_surge_days)

### Largest Price Drop Day

In [ ]:
# Ensure the Year column exists
df['Year'] = df.index.year

# Get the index of the lowest daily return in each year
date_drop_min = df_ret.groupby('Year')['SimpleDailyReturns'].idxmin()

# Retrieve those rows
largest_drop_days = df.loc[date_drop_min, ['Year', 'SimpleDailyReturns']]

# Display the results
print("Largest Price Drop Day per Year:")
print(largest_drop_days)

### Highest Volume Day

In [ ]:
df['Year'] = df.index.year

unique_years = sorted(df['Year'].unique())
print(f"Unique years in dataset: {unique_years}")
# Years range from 2020 to 2025; we should therefore obtain 6 dates with the date and max volume within each year

# To find the date where the Volume is maximised (for each year)
date_Vol_max = df.groupby('Year')['Volume'].idxmax()

highest_Vol_year = df.loc[date_Vol_max]

print(highest_Vol_year[['Year', 'Volume']])

### Highest High and Lowest Low

The results below illustrate the dates and values of DoorDash's highest and lowest stock prices per year since IPO.

In [ ]:
# Highest High (per year)
date_High_max = df.groupby('Year')['High'].idxmax()

# Lowest Low (per year)
date_Low_min = df.groupby('Year')['Low'].idxmin()

highest_high_year = df.loc[date_High_max]
lowest_low_year = df.loc[date_Low_min]

print("Highest High Each Year:")
print(highest_high_year[['Year', 'High']])

print("\nLowest Low Each Year:")
print(lowest_low_year[['Year', 'Low']])

### Annual Open and Close

The results below show DoorDash's opening (at the start of each year) and close prices (at the end of each year).


Including the exact date is optional as the *Open* must be the first trading day of the year (e.g., 2nd Jan), and the *Close* must be the last trading day.

In [ ]:
date_Open_first = df.groupby('Year').head(1).index
date_Close_last = df.groupby('Year').tail(1).index

annual_open = df.loc[date_Open_first, ['Year', 'Open']]
annual_close = df.loc[date_Close_last, ['Year', 'Close']]

print("Annual Open Prices:")
print(annual_open.to_string(index = False))

print("\nAnnual Close Prices:")
print(annual_close.to_string(index = False))

## Data Visualisation

### Line Trend Chart
**Close Price with SMA and Bollinger Bands**

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(df.index, df["Close"], label="Close", color="blue", linewidth=1)
plt.plot(df.index, df["20Day_SMA"], label="20-Day SMA", color="orange", linewidth=1)
plt.fill_between(df.index, df["Upper_BB"], df["Lower_BB"], color="gray", alpha=0.5, label="Bollinger Bands")
plt.title("Close Price with SMA and Bollinger Bands")
plt.xlabel("Date")
plt.ylabel("Price(USD)")
plt.legend()
plt.tight_layout()
plt.show()

**Volume and Volatility Subplots**

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
ax1.plot(df.index, df["Volume"], color="purple")
ax1.set_title("Daily Trading Volume")
ax1.set_ylabel("Volume")
ax2.plot(df.index, df["20Day_Volatility"], color="green")
ax2.set_title("20-Day Rolling Volatility")
ax2.set_ylabel("Volatility")
plt.xlabel("Date")
plt.tight_layout()
plt.show()

### Histogram Chart

**Histogram of Log Returns**

In [ ]:
recommended_bins = int(np.sqrt(len(df.Log_Returns)))
recommended_bins # is 33

plt.figure(figsize=(10, 7))
plt.hist(df.Log_Returns, bins=33)
plt.title("Histogram of Log Returns")
plt.ylabel("Frequency")
plt.xlabel("Log Returns")
plt.show()


import seaborn as sns
plt.figure(figsize=(10, 5))
sns.histplot(df["Log_Returns"].dropna(), bins=50, kde=True, color='teal')
plt.title("Histogram of Log Returns")
plt.xlabel("Log Return")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


### Scatter Plot Chart

**Scatter Plot: Volume vs Daily Return**

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(df.SimpleDailyReturns, df.Volume)
plt.title("Volume vs Daily Returns")
plt.ylabel("Volume")
plt.xlabel("Simple Daily Return")
plt.tight_layout()
plt.show()